<a href="https://colab.research.google.com/github/Mishra-0709/ML-MODELS/blob/main/ColourTheoryMyntra.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Data Preparation

In [2]:
import numpy as np
import pandas as pd

# Load the trainval.txt and test.txt files
trainval_data = pd.read_csv('trainval.txt', header=None, names=['id'])
test_data = pd.read_csv('test.txt', header=None, names=['id'])

# Load the labels.txt file
with open('labels.txt', 'r') as file:
    labels = file.read().split(',')

# Create a mapping from IDs to labels (for demonstration purposes)
# This mapping is assumed; you would have actual label data in a real scenario
np.random.seed(42)
trainval_data['label'] = np.random.choice(labels, len(trainval_data))
test_data['label'] = np.random.choice(labels, len(test_data))

# Display the data
print(trainval_data.head())
print(test_data.head())

       id    label
0  168468  \nskirt
1   20751  \nshirt
2  184291  \ndress
3   63239  \npants
4  110606  \nskirt
       id    label
0  185218  \npants
1   45639    \nhat
2     449  \ndress
3   33441  \npants
4   87956    \nhat


Skin Tone Detection

In [5]:
import tensorflow as tf
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img, img_to_array
from tensorflow.keras.utils import Sequence
import os
import numpy as np

# Load labels from labels.txt
with open('labels.txt') as f:
    labels = f.read().split(',')
num_classes = len(labels)

# Load training and validation IDs from trainval.txt
with open('trainval.txt') as f:
    trainval_ids = f.read().splitlines()

# Load test IDs from test.txt
with open('test.txt') as f:
    test_ids = f.read().splitlines()

# Define the paths to the image directories (update these paths as necessary)
trainval_dir = 'path_to_trainval_images'
test_dir = 'path_to_test_images'

# Print statements for debugging
print(f"Trainval directory: {trainval_dir}")
print(f"Test directory: {test_dir}")
print(f"Trainval directory exists: {os.path.exists(trainval_dir)}")
print(f"Test directory exists: {os.path.exists(test_dir)}")

# Custom data generator
class CustomDataGenerator(Sequence):
    def __init__(self, image_dir, image_ids, labels, batch_size=32, target_size=(224, 224), is_training=True, validation_split=0.2):
        self.image_dir = image_dir
        self.image_ids = image_ids
        self.labels = labels
        self.batch_size = batch_size
        self.target_size = target_size
        self.is_training = is_training
        self.validation_split = validation_split
        self.image_data_generator = ImageDataGenerator(rescale=1./255, validation_split=validation_split)

        if is_training:
            self.image_ids = self.image_ids[:int(len(self.image_ids) * (1 - validation_split))]
        else:
            self.image_ids = self.image_ids[int(len(self.image_ids) * (1 - validation_split)):]

    def __len__(self):
        return int(np.ceil(len(self.image_ids) / self.batch_size))

    def __getitem__(self, index):
        batch_ids = self.image_ids[index * self.batch_size:(index + 1) * self.batch_size]
        images = []
        image_labels = []

        for img_id in batch_ids:
            img_path = os.path.join(self.image_dir, f"{img_id}.jpg")
            if os.path.exists(img_path):
                img = load_img(img_path, target_size=self.target_size)
                img = img_to_array(img) / 255.0
                images.append(img)
                img_label_index = self.labels.index(img_id.split('_')[0])  # Assuming label is part of the file name
                image_labels.append(tf.keras.utils.to_categorical(img_label_index, num_classes=num_classes))

        return np.array(images), np.array(image_labels)

# Create the data generators
train_generator = CustomDataGenerator(trainval_dir, trainval_ids, labels, is_training=True)
validation_generator = CustomDataGenerator(trainval_dir, trainval_ids, labels, is_training=False)

# Load the pre-trained MobileNetV2 model, excluding the top layers
base_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Add new top layers for classification
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
predictions = Dense(num_classes, activation='softmax')(x)

# Create the full model
model = Model(inputs=base_model.input, outputs=predictions)

# Freeze the base model layers
for layer in base_model.layers:
    layer.trainable = False

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Save the model
model.save('skin_tone_model.h5')

Trainval directory: path_to_trainval_images
Test directory: path_to_test_images
Trainval directory exists: False
Test directory exists: False


/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


Color Theory Application

In [6]:
# Define a color palette for each skin tone category
color_palettes = {
    'light': ['pastel pink', 'light blue', 'mint green'],
    'medium': ['peach', 'olive green', 'mustard'],
    'dark': ['deep red', 'royal blue', 'emerald green']
}

# Function to get recommended colors based on skin tone
def get_recommended_colors(skin_tone):
    return color_palettes.get(skin_tone, ['default color'])

# Example usage
skin_tone = 'medium'
recommended_colors = get_recommended_colors(skin_tone)
print(f'Recommended colors for {skin_tone} skin tone: {recommended_colors}')

Recommended colors for medium skin tone: ['peach', 'olive green', 'mustard']


Personalized Recommendation Engine

In [7]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel

# Sample user-item interaction data (for collaborative filtering)
user_item_data = pd.DataFrame({
    'user_id': [1, 1, 2, 2, 3, 3],
    'item_id': [101, 102, 101, 103, 102, 104],
    'rating': [5, 4, 5, 4, 4, 5]
})

# Sample item metadata (for content-based filtering)
item_metadata = pd.DataFrame({
    'item_id': [101, 102, 103, 104],
    'description': ['red dress', 'blue jeans', 'green shirt', 'yellow skirt']
})

# Collaborative filtering (using user-item interactions)
user_item_matrix = user_item_data.pivot(index='user_id', columns='item_id', values='rating').fillna(0)

# Content-based filtering (using item descriptions)
tf

<module 'tensorflow' from '/usr/local/lib/python3.10/dist-packages/tensorflow/__init__.py'>